<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-Buckey-and-Upload-to-AWS-S3" data-toc-modified-id="Create-Buckey-and-Upload-to-AWS-S3-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create Buckey and Upload to AWS S3</a></span><ul class="toc-item"><li><span><a href="#Creat-Bucket" data-toc-modified-id="Creat-Bucket-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Creat Bucket</a></span></li><li><span><a href="#Upload-File" data-toc-modified-id="Upload-File-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Upload File</a></span></li></ul></li><li><span><a href="#Run-the-file-through-AWS-Rekognition" data-toc-modified-id="Run-the-file-through-AWS-Rekognition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run the file through AWS Rekognition</a></span><ul class="toc-item"><li><span><a href="#Start-label-detection-and-get-result" data-toc-modified-id="Start-label-detection-and-get-result-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Start label detection and get result</a></span></li></ul></li></ul></div>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import boto3 # AWS client
import os # File management

# data management
import numpy as np
import pandas as pd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# small tools
import time
from sklearn.preprocessing import StandardScaler
import random
import math
from statistics import mean

**Pre-requiests**
- aws cli installed
by "pip install awscli"
- IAM user configured

Access key ID  
Secret Access Key  
Default Region Name  
Default output form: json

# Create Buckey and Upload to AWS S3

In [3]:
s3_client = boto3.client('s3',region_name='eu-west-1')

## Creat Bucket

In [4]:
response = s3_client.list_buckets()

In [5]:
bucket_name = '12-sample-videos'

In [6]:
list_of_buckets = []
for bucket in response['Buckets']:
    list_of_buckets.append(bucket["Name"])
print(list_of_buckets)
if bucket_name not in list_of_buckets:
    response = s3_client.create_bucket(
        ACL = 'private',
        Bucket = '12-sample-videos',
        CreateBucketConfiguration={'LocationConstraint': 'eu-west-1'}
    )  
    print('bukcet created')
else: 
    print('bukcet already exists')

['12-sample-videos']
bukcet already exists


## Upload File

In [7]:
# these are our video data
files = os.listdir('data')
random.sample(files, 5)

['02_Deep Clean_RouteTwo_B_V2.mp4',
 'black_friday_gif.mp4',
 'christmas_proform (1).mp4',
 '15- edit Handy Andy Cream.mp4',
 'skip_bucket_or.mp4']

In [8]:
files

['01_Deep Clean_RouteOne_V2.mp4',
 '02_Deep Clean_RouteTwo_B_V2.mp4',
 '03-Deep Clean_RouteThree_V2.mp4',
 '03_Deep Clean_Asset 3.mp4',
 '15- edit Domestos.mp4',
 '15- edit Handy Andy Cream.mp4',
 '15- edit Sunlight Glass.mp4',
 '15- edit Sunlight Pots.mp4',
 '333984_prs_lt_product-test-ce13_vid15_1080x1080_en.mp4',
 '346216_prs_lt_275966-renovate-and-repair-msl-op63-fail_vid15_1080x1350_en.mp4',
 '3in1_ceramic_or_sf.mp4',
 'black_friday_gif.mp4',
 'Cartoon 16x9 EDSF CTA 2.mp4',
 'Cartoon 16x9 EDSF CTA.mp4',
 'Cartoon 4x5 EDSF.mp4',
 'Cartoon 9x16 EDSF CTA.mp4',
 'Cartoon 9x16 EDSF NO CTA.mp4',
 'Cartoon 9x16 TikTok EDSF NO CTA.mp4',
 'christmas_proform (1).mp4',
 'christmas_proform (2).mp4',
 'christmas_proform.mp4',
 'cleaningguide_1080x1080 2.mp4',
 'cleaningguide_1080x1080.mp4',
 'flu_season_video_2.mp4',
 'Hand Hygiene Video BE FR.mp4',
 'highfive_ss.mp4',
 'iPhoneiOS13_ThingsYouCanSay_v2.mp4',
 'j00525uk_uni_upro_occ_hand_hygiene_1x1.mp4',
 'kr1013278kr_11910_bw3_lt_r29-small-bub

In [9]:
n_files = len(files)
n_files

42

In [10]:
# list objects in the bucket
response = s3_client.list_objects(Bucket=bucket_name)
objects_in_bucket = []
# print out the content name
for res in response['Contents']:
    objects_in_bucket.append(res['Key'])

In [11]:
# check how many items are there now
len(objects_in_bucket)

42

In [12]:
# upload the file one by one, using its original name as the file name in S3
# if the file already exists in the bucket, skip it
for n in range(n_files):
    if files[n] not in objects_in_bucket:
        path = 'data/' + files[n]
        response = s3_client.upload_file(path, bucket_name, files[n])

In [13]:
# list objects in the bucket
response = s3_client.list_objects(Bucket=bucket_name)
objects_in_bucket_new = []
# print out the content name
for res in response['Contents']:
    objects_in_bucket_new.append(res['Key'])

In [14]:
# check how many items are there now
len(objects_in_bucket_new)

42

In [15]:
# to download file
# response = s3_client.download_file('which file', 'download_as_name', r'download_to_path')

# Run the file through AWS Rekognition 

In [16]:
rek_client = boto3.client('rekognition')

The video must be stored in an Amazon S3 bucket. Use Video to specify the bucket name and the filename of the video. StartLabelDetection returns a job identifier (JobId) which you use to get the results of the operation.

## Start label detection and get result

In [17]:
# the free credits have a limitation of 20 jobs
# thus, we will only run 20 files each time and then combine the results together 
job_id_list = []
for n in range(20):    
    # start label detection
    response = rek_client.start_label_detection(MinConfidence = 80,
                                                Video={'S3Object':{'Bucket':bucket_name,
                                                                   'Name':files[n]}})
    job_id_list.append(response['JobId'])

In [19]:
# here you have to pause for a while
# since the maximum of taks that AWS can process at once is 20

for n in range(20, 40):
    # start label detection
    response = rek_client.start_label_detection(MinConfidence = 80,
                                                Video={'S3Object':{'Bucket':bucket_name,
                                                                   'Name':files[n]}})
    job_id_list.append(response['JobId'])

In [20]:
for n in range(40, n_files):
    # start label detection
    response = rek_client.start_label_detection(MinConfidence = 80,
                                                Video={'S3Object':{'Bucket':bucket_name,
                                                                   'Name':files[n]}})
    job_id_list.append(response['JobId'])

In [21]:
len(job_id_list)

43

In [22]:
# The result for each job id would probably be saved for 24 hours (for some services it's 7 days)
# thus we would want to keep an record of these ids
with open('job_ids.txt','w') as f:
    f.write('\n'.join(job_id_list))

**Wait for a while**

In [23]:
result_list = []
# wait for a little while
for n in range(len(files)):      
    result = rek_client.get_label_detection(JobId=job_id_list[n],
                                            SortBy='TIMESTAMP',
                                           MaxResults = 5000)
    result_list.append(result)

In [24]:
count = 0
for n in range(len(files)):
    if result_list[n]['JobStatus'] == 'SUCCEEDED':
        count += 1
print('%s out of %s tasks completed' % (count, n_files))

42 out of 42 tasks completed


We would regard each appearence of the label as success since the minimum confidence is set high. Therefore, we could count the number of appearence.

**Save the label counts locally**

In [25]:
list_labels = []
for result in result_list:
    n_instances = len(result['Labels'])
    # print(n_instances)
    labels = []
    for instance in result['Labels']:
        labels.append(instance['Label']['Name'])
    list_labels.append(labels)

In [26]:
unique_label = np.unique([item for sublist in list_labels for item in sublist])

In [27]:
len(unique_label)

224

In [28]:
# prepare an empty dataframe
df = pd.DataFrame(np.zeros((len(files),len(unique_label))),
                  columns = unique_label)

In [29]:
df.shape

(42, 224)

In [30]:
df.head(2)

,Accessories,Adult,Advertisement,Airport,Alcohol,Alien,Aluminium,Animal,Appliance,Architecture,...,Vest,Washer,Washing,Water,Weapon,Wedding,Wine,Wine Glass,Woman,Wood
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Put the count value to the df
for n in range(len(list_labels)):
    unique, counts = np.unique(list_labels[n], return_counts=True)
    for n2 in range(len(unique)):
        word = unique[n2]
        df[word].iloc[n] = counts[n2]

In [33]:
df.head(2)

,Accessories,Adult,Advertisement,Airport,Alcohol,Alien,Aluminium,Animal,Appliance,Architecture,...,Vest,Washer,Washing,Water,Weapon,Wedding,Wine,Wine Glass,Woman,Wood
0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,21.0,0.0
1,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# add content length to the df
content_lengths = []
for result in result_list:
    content_length = result['ResponseMetadata']['HTTPHeaders']['content-length']
    content_lengths.append(content_length)

In [34]:
content_lengths = np.array(content_lengths, dtype=int)
content_lengths[0:5]

array([53280, 13007, 12057, 11262, 61387])

In [35]:
len(content_lengths)

42

In [36]:
df['content_lengths'] = content_lengths

In [37]:
df.shape

(42, 225)

In [38]:
# save this df of count
# df.to_csv('sample_video_label_count.csv',index=False)